# Semantic map

Compress all points at once

In [2]:
import numpy as np
import cv2
from plyfile import PlyData

# Load the PLY file
ply_file_path = "/home/yuanyan/autonomous-exploration-with-lio-sam/maps/office_segmented_no_ceiling.ply"  # Replace with your PLY file path
ply_data = PlyData.read(ply_file_path)

# Extract all vertices and colors
vertices = np.array([
    ply_data['vertex']['x'],
    ply_data['vertex']['y'],
    ply_data['vertex']['z']
]).T

colors = np.array([
    ply_data['vertex']['red'],
    ply_data['vertex']['green'],
    ply_data['vertex']['blue']
]).T

# If there are no points, exit early
if len(vertices) == 0:
    print("No points found in the point cloud.")
    exit(0)

# --- 1. Compute bounding box in the X and Y dimensions ---
min_vals = np.min(vertices, axis=0)
max_vals = np.max(vertices, axis=0)

cloud_width = max_vals[0] - min_vals[0]   # Range in X
cloud_height = max_vals[1] - min_vals[1] # Range in Y

# Handle degenerate case: if cloud_height is 0 (all points have the same Y)
if cloud_height == 0:
    cloud_height = 1e-6  # Avoid division by zero

# --- 2. Define image size and scaling ---
image_height = 500
scale_factor = image_height / cloud_height
image_width = int(cloud_width * scale_factor)
if image_width <= 0:
    image_width = 1

# --- 3. Normalize and scale point coordinates to image space ---
# Shift min_vals to origin, then scale
normalized_points = (vertices[:, :2] - min_vals[:2]) * scale_factor
normalized_points = normalized_points.astype(np.int32)  # convert to integer pixel coords

# --- 4. Create a blank color image (BGR in OpenCV) ---
color_image = np.zeros((image_height, image_width, 3), dtype=np.uint8)

# --- 5. Project points onto the image, setting each pixel’s color ---
for i in range(len(vertices)):
    x, y = normalized_points[i]
    
    # Clip to image boundaries
    if x < 0 or x >= image_width or y < 0 or y >= image_height:
        continue
    
    # OpenCV uses BGR ordering, so reorder the color array
    # Original color is [R, G, B], we convert to [B, G, R]
    r, g, b = colors[i]
    color_image[y, x] = [b, g, r]

# --- 6. Display or save the result ---
cv2.imshow("Point Cloud Projection", color_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# If you want to save:
# cv2.imwrite("cloud_projection.png", color_image)



Compress each class seperately

In [ ]:
import numpy as np
import cv2
from plyfile import PlyData

# Load the PLY file
ply_file_path = "/home/yuanyan/autonomous-exploration-with-lio-sam/maps/office_segmented_no_ceiling.ply"  # Replace with your PLY file path
ply_data = PlyData.read(ply_file_path)

# Extract vertices and colors
vertices = np.array([
    ply_data['vertex']['x'],
    ply_data['vertex']['y'],
    ply_data['vertex']['z']
]).T

colors = np.array([
    ply_data['vertex']['red'],
    ply_data['vertex']['green'],
    ply_data['vertex']['blue']
]).T

# Class names and corresponding colors
class_names = ['ceiling', 'floor', 'wall', 'beam', 'column', 'window', 'door', 'chair', 'table', 'bookcase', 'sofa', 'board', 'clutter', 'ignored']
class_colors = np.array([
    [233, 229, 107],
    [ 95, 156, 196],
    [179, 116,  81],
    [241, 149, 131],
    [ 81, 163, 148],
    [ 77, 174,  84],
    [108, 135,  75],
    [ 41,  49, 101],
    [ 79,  79,  76],
    [223,  52,  52],
    [ 89,  47,  95],
    [ 81, 109, 114],
    [233, 233, 229],
    [  0,   0,   0]
])

# --- 1. Compute bounding box in the X and Y dimensions ---
min_vals = np.min(vertices, axis=0)
max_vals = np.max(vertices, axis=0)

cloud_width = max_vals[0] - min_vals[0]   # Range in X
cloud_height = max_vals[1] - min_vals[1] # Range in Y

# Handle degenerate case: if cloud_height is 0 (all points have the same Y)
if cloud_height == 0:
    cloud_height = 1e-6  # Avoid division by zero

# --- 2. Define image size and scaling ---
image_height = 500
scale_factor = image_height / cloud_height
image_width = int(cloud_width * scale_factor)
if image_width <= 0:
    image_width = 1

# --- 3. Normalize and scale point coordinates to image space ---
# Shift min_vals to origin, then scale
normalized_points = (vertices[:, :2] - min_vals[:2]) * scale_factor
normalized_points = normalized_points.astype(np.int32)  # convert to integer pixel coords

# --- 4. Create individual images for each class ---
class_images = {name: np.zeros((image_height, image_width, 3), dtype=np.uint8) for name in class_names}

# --- 5. Project points onto the images by class ---
for i in range(len(vertices)):
    x, y = normalized_points[i]

    # Clip to image boundaries
    if x < 0 or x >= image_width or y < 0 or y >= image_height:
        continue

    # Match the point's color to a class
    for class_idx, class_color in enumerate(class_colors):
        if np.array_equal(colors[i], class_color):
            class_images[class_names[class_idx]][y, x] = class_color
            break

# --- 6. Apply morphological operations to densify points ---
kernel = np.ones((5, 5), np.uint8)  # Kernel size for erosion
for class_name, class_color in zip(class_names, class_colors):
    # Convert the image to a binary mask using non-zero pixel detection
    binary_image = cv2.inRange(class_images[class_name], (1, 1, 1), (255, 255, 255))
    
    # Apply dilation to densify the points
    dilation_image = cv2.dilate(binary_image, kernel, iterations=8)
    
    # Set the dilation image pixels to the corresponding class color
    dilation_colored = np.zeros_like(class_images[class_name])
    dilation_colored[np.where(dilation_image > 0)] = class_color
    class_images[class_name] = dilation_colored

# --- 7. Perform polygon detection and overlay based on area ---
polygon_classes = ['wall', 'beam', 'column', 'window', 'door', 'chair', 'table', 'board', 'clutter']
combined_image = np.zeros((image_height, image_width, 3), dtype=np.uint8)
overlay_mask = np.zeros((image_height, image_width), dtype=np.float32)  # Store area of polygons per pixel
polygon_vertices = []  # Store remapped polygon vertices

for class_name, class_color in zip(class_names, class_colors):
    if class_name not in polygon_classes:
        continue

    # Convert the image to a binary mask
    binary_image = cv2.inRange(class_images[class_name], (1, 1, 1), (255, 255, 255))

    # Find contours (polygons)
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Compute the polygon's area
        area = cv2.contourArea(contour)

        # Create a mask for the polygon
        polygon_mask = np.zeros((image_height, image_width), dtype=np.uint8)
        cv2.drawContours(polygon_mask, [contour], -1, 255, thickness=cv2.FILLED)

        # Update the combined image based on overlay mask
        for y in range(image_height):
            for x in range(image_width):
                if polygon_mask[y, x] == 255:
                    if overlay_mask[y, x] == 0 or area < overlay_mask[y, x]:
                        combined_image[y, x] = class_color
                        overlay_mask[y, x] = area

        # Add labels to the polygons
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])  # Centroid x
            cy = int(M['m01'] / M['m00'])  # Centroid y
            cv2.putText(combined_image, class_name, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

        # Remap polygon vertices to point cloud frame
        remapped_contour = []
        for point in contour:
            x_img, y_img = point[0]  # Contour points are (x, y) in image space
            x_pc = x_img / scale_factor + min_vals[0]  # Map back to point cloud X
            y_pc = y_img / scale_factor + min_vals[1]  # Map back to point cloud Y
            remapped_contour.append([x_pc, y_pc])
        polygon_vertices.append(np.array(remapped_contour))

# --- 8. Display and save the final combined image ---
cv2.imshow("Combined Image with Polygons", combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

for i, vertices in enumerate(polygon_vertices):
    print(f"Polygon {i+1} vertices in point cloud frame:")
    print(vertices)
